# Importing packages from the environment into the Notebook

In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [3]:
import pandas as pd 

In [4]:
df = pd.read_csv('real_estate_db.csv', encoding='latin-1') # imports the csv into the local notebook
df.head()

,UID,BLOCKID,SUMLEVEL,COUNTYID,STATEID,state,state_ab,city,place,type,...,female_age_mean,female_age_median,female_age_stdev,female_age_sample_weight,female_age_samples,pct_own,married,married_snp,separated,divorced
0,220336,NaN,140,16,2,Alaska,AK,Unalaska,Unalaska City,City,...,32.78177,31.91667,19.31875,440.46429,1894.0,0.25053,0.47388,0.30134,0.03443,0.09802
1,220342,NaN,140,20,2,Alaska,AK,Eagle River,Anchorage,City,...,38.97956,39.66667,20.05513,466.65478,1947.0,0.94989,0.52381,0.01777,0.00782,0.13575
2,220343,NaN,140,20,2,Alaska,AK,Jber,Anchorage,City,...,22.20427,23.16667,13.86575,887.67805,3570.0,0.00759,0.50459,0.06676,0.01000,0.01838
3,220345,NaN,140,20,2,Alaska,AK,Anchorage,Point Mackenzie,City,...,37.00750,34.00000,22.06347,281.49420,1049.0,0.20247,0.44428,0.05933,0.00000,0.21563
4,220347,NaN,140,20,2,Alaska,AK,Anchorage,Anchorage,City,...,34.96611,31.75000,20.49887,655.98066,2905.0,0.56936,0.51034,0.08315,0.06731,0.08711


# Cleaning the data

In [5]:
df.drop_duplicates(subset=None, inplace=True) # drops duplicate rows
df.drop('BLOCKID', axis=1, inplace=True) # drops the nan-filled column

In [ ]:
df = df.replace(r'^\s*$', np.nan, regex=True) # turns null characters into nan
df.dropna(axis=0, inplace=True) # drops nan-filled columns

In [ ]:
df_sampled = df.sample(frac=0.01) # samples 0.1 of the db.

In [ ]:
df_sampled.head() # prints the first five indexes of the new sampled db.

# Function to turn hs_degree into a binary

In [ ]:
# finds and prints the mean rate of hs_degree
hs_degree_mean = df_sampled[['hs_degree']].mean()
print(hs_degree_mean)

In [ ]:
# write function to check whether or not the average person is graduated
def check_condition(value, threshold):
    if value > threshold:
        return 'Yes'
    else:
        return 'No'

# uses function to check whether values are greater or lesser and writes to new column
df_sampled['avg_person_graduated'] = df_sampled['hs_degree'].apply(lambda x: check_condition(x, 0.855329))

# verifies the counts of yesses and nos within the newly generated column
yes_count = df_sampled[df_sampled['avg_person_graduated'] == 'Yes'].shape[0]
no_count = df_sampled[df_sampled['avg_person_graduated'] == 'No'].shape[0]

# verifies whether function worked
print(f"\nNumber of 'Yes' in avg_person_graduated: {yes_count}")
print(f"Number of 'No' in avg_person_graduated: {no_count}")
df_sampled.head()

# Neural Network

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# defines x and y values for training
data_no_hs_degree = df_sampled[['debt', 'rent_mean', 'hc_mortgage_mean', 'home_equity_second_mortgage', 'second_mortgage', 'home_equity', 'family_mean']]
X = data_no_hs_degree
y = df_sampled['avg_person_graduated'].apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
# splits data into x and y trains and preds
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
print(X_train) # prints X_train

In [ ]:
print(y_train) # prints y_train

In [ ]:
np.isnan(X_train).any() # checks X_train for any nan values

In [ ]:
X_train.isnull().any() # checks X_train for any null values

In [ ]:
np.isnan(y_train).any() # checks y_train for any nan values

In [ ]:
y_train.isnull().any() # checks y_train for any null values

In [ ]:
# loads important models
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.metrics import accuracy_score

# defines model, including two layers
model = Sequential()
model.add(Dense(units=32, activation='ReLU', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='ReLU'))
model.add(Dense(units=1, activation='sigmoid'))

#imports optimizer and learning rate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

# optimizer
initial_learning_rate = 0.001  # learning rate
optimizer = Adam(learning_rate=initial_learning_rate)

# learning rate scheduler
def lr_scheduler(epoch):
    return initial_learning_rate * 0.95 ** epoch

# callbacks learning rate per epoch
lr_callback = LearningRateScheduler(lr_scheduler)

In [ ]:
# defines model compiler
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# fits model
model.fit(X_train, y_train, batch_size = 1, epochs = 100, callbacks=[lr_callback])

In [ ]:
# performs a prediction
y_hat = model.predict(X_test)

# prints out the whole accuracy
print(y_hat)

# reevaluates to binary
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

# prints out the accuracy again
print(y_hat)

# prints final accuracy
print(accuracy_score(y_test, y_hat))

In [ ]:
accuracy_score(y_test, y_hat)

# Heatmap

In [ ]:
import matplotlib.pyplot as mp 
import seaborn as sb 
from PIL import _imaging
from PIL import Image
  
data = df_sampled[['hs_degree', 'hs_degree_male', 'hs_degree_female', 'debt', 'rent_mean', 'hc_mortgage_mean', 'home_equity_second_mortgage', 'second_mortgage', 'home_equity', 'family_mean']]
 
print(data.corr()) 
  
dataplot=sb.heatmap(data.corr(), annot=True)
  
mp.show() 

# Interactive US Map

In [ ]:
import plotly.express as px
avg_grad_rates = df.groupby('state_ab')['hs_degree'].mean().reset_index()

fig = px.choropleth(avg_grad_rates,
                    locations='state_ab',
                    locationmode='USA-states',
                    color='hs_degree',
                    hover_name='state_ab',
                    color_continuous_scale='blues',
                    title='Average High School Graduation Rates by State')
fig.update_layout(geo_scope='usa')
fig.show()

# Scatterplot of Degree Percentage versus Debt Percentage

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(df_sampled['debt'], df_sampled['hs_degree'], color='b', alpha=0.5)
plt.title('High School Degree Percentage vs Debt Percentage')
plt.xlabel('Debt Percentage')
plt.ylabel('High School Degree Percentage')
plt.grid(True)
plt.show()

# Line Chart of Graduation Rate versus Debt Rate

In [ ]:
bins = np.linspace(0, 1, 11)  # Bins from 0 to 1 with 5 intervals
labels = [f'{bins[i]:.2f}-{bins[i+1]:.2f}' for i in range(len(bins)-1)]

df['debt_binned'] = pd.cut(df['debt'], bins=bins, labels=labels, right=False)

avg_graduation_rate = df.groupby('debt_binned')['hs_degree'].mean()

plt.figure(figsize=(10, 6))
plt.plot(avg_graduation_rate.index, avg_graduation_rate.values, marker='o', linestyle='-', color='skyblue')

plt.xlabel('Debt Percentage Range')
plt.ylabel('Average Graduation Rate')
plt.title('Average Graduation Rate by Debt Percentage Range')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Random Forest Classifier

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# identify the x and y
X = df_sampled[['debt', 'family_mean']]
y = df_sampled['avg_person_graduated'].apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# creates the random forest classifier
clf = RandomForestClassifier(random_state=42)

In [ ]:
# trains the model
clf.fit(X_train, y_train)

In [ ]:
# predicts values based on previously split test set
y_pred = clf.predict(X_test)

In [ ]:
# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
# plots boundaries
if X.shape[1] == 2:
    plt.figure(figsize=(8, 6))
    xx, yy = np.meshgrid(np.linspace(X.values[:, 0].min() - 0.1, X.values[:, 0].max() + 0.1, 100),
                         np.linspace(X.values[:, 1].min() - 100, X.values[:, 1].max() + 100, 100))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    plt.scatter(X.values[:, 0], X.values[:, 1], c=y, cmap='viridis', edgecolors='k')
    plt.xlabel('Debt')
    plt.ylabel('Family Income Mean')
    plt.title('Random Forest Classifier Decision Boundaries')
    plt.colorbar()
    plt.show()

# plots features
plt.figure(figsize=(8, 6))
plt.bar(X.columns, clf.feature_importances_, color='skyblue')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance of Random Forest Classifier')
plt.show()

# Decision Tree Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score

In [ ]:
# find x and y
X = df_sampled[['debt', 'family_mean']]
y = df_sampled['avg_person_graduated'].apply(lambda x: 1 if x=='Yes' else 0)

In [ ]:
# separates data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# creates a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)

In [ ]:
# trains the classifier
clf.fit(X_train, y_train)

In [ ]:
# predicts x_test
y_pred = clf.predict(X_test)

In [ ]:
# accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

In [ ]:
# plots the boundaries
plt.figure(figsize=(8, 6))

# plots points of each class
plt.scatter(X[y == 0]['debt'], X[y == 0]['family_mean'], cmap='viridis', edgecolors='k', label='Not Graduated')
plt.scatter(X[y == 1]['debt'], X[y == 1]['family_mean'], cmap='viridis', edgecolors='k', label='Graduated')

# creates a meshgrid to plot the decision boundaries
xx, yy = np.meshgrid(np.linspace(X['debt'].min() - 0.1, X['debt'].max() + 0.1, 100),
                     np.linspace(X['family_mean'].min() - 100, X['family_mean'].max() + 100, 100))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

# plots the contour filled regions
plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')

# labels and title
plt.xlabel('Debt')
plt.ylabel('Family Income Mean')
plt.title('Decision Tree Classifier Decision Boundaries')
plt.colorbar()
plt.legend()
plt.show()
# plots the decision tree
plt.figure(figsize=(12, 8))
plot_tree(clf, filled=True, feature_names=X.columns, class_names=['Not Graduated', 'Graduated'])
plt.title('Decision Tree Classifier')
plt.show()

# Pair Plot of Rent Mean versus Highschool Degree Rate

In [ ]:
sb.pairplot(df_sampled[['rent_mean', 'hs_degree']], height=3)
plt.suptitle('Pair Plot: Rent Mean vs HS Degree', y=1.02)
plt.show()

# Joint Plot of Mortgage Mean versus High School Degree Rate

In [ ]:
sb.jointplot(x='hc_mortgage_mean', y='hs_degree', data=df_sampled, kind='hex', height=6)
plt.suptitle('Joint Plot (Hexbin): Household Mortgage Mean vs HS Degree', y=1.02)
plt.show()

# Scatter Plot of Family Income Mean versus High School Degree Rate

In [ ]:
sb.lmplot(x='family_mean', y='hs_degree', data=df_sampled, height=6, aspect=1.5, scatter_kws={'alpha':0.5})
plt.title('Scatter Plot with Regression Line: Family Income Mean vs HS Degree')
plt.show()